In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

In [2]:
#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
rnn01 = pd.read_csv(path+'../sub/rnnCV_0106A.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [ ]:
#rnn27trn = pd.read_pickle('../../features/y_pred_trn.pkl')
#rnn27tst = pd.read_csv('../rnndhsub_2705A.csv.gz', compression='gzip')
#rnn27 = pd.concat([rnn27tst, rnn27trn.reset_index().rename(columns = {0:'deal_probability'})], axis = 0)
#rnn27.to_csv('../rnnCV_2705A.csv.gz', compression='gzip')
#rnn01A = pd.read_csv('../rnnCV_0106A_tmp.csv.gz', compression='gzip')
#rnn01B = pd.read_csv('../rnnCV_0106A_tmp1.csv.gz', compression='gzip')
#idx = rnn01A['rnn_preds']==0
#rnn01A[idx] = rnn01B[idx] 
#rnn01A.to_csv(path+'../sub/rnnCV_0106A.csv.gz', compression='gzip', index =  False)

In [5]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
rnn01.rename(columns={'rnn_preds': 'rnn01_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(rnn01, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [6]:
preds_df['difference_l02_r']=preds_df['lgb02_preds'] - preds_df['rnn_preds']
preds_df['difference_l02_r1']=preds_df['lgb02_preds'] - preds_df['rnn27_preds']
preds_df['difference_l02_r2']=preds_df['lgb02_preds'] - preds_df['rnn01_preds']
preds_df['difference_l02_m']=preds_df['lgb02_preds'] - preds_df['mlp_preds']
preds_df['difference_r_m']=preds_df['rnn27_preds'] - preds_df['mlp_preds']
#preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
preds_df['price'].fillna(-1,inplace=True)

preds_df['max'] = np.max(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in preds_df.columns if '_preds' in col]),axis=0)

In [7]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,rnn01_preds,mlp_preds,lgb31_preds,lgb02_preds,Unnamed: 0,rnn27_preds,user_id,...,deal_probability,difference_l02_r,difference_l02_r1,difference_l02_r2,difference_l02_m,difference_r_m,max,min,avg,std
0,b912c3c6a6ad,0.091656,0.069891,0.065340,0.091223,0.083219,0.090855,0,0.068331,e00f8ff2eaf9,...,0.12789,0.020964,0.022525,0.025516,-0.000368,-0.022892,0.091656,0.065340,0.080074,0.010976
1,2dac0150717d,0.147801,0.075968,0.057082,0.211844,0.157973,0.119609,1,0.070638,39aeb48f0017,...,0.00000,0.043641,0.048971,0.062526,-0.092236,-0.141206,0.211844,0.057082,0.120131,0.052077
2,ba83aefab5dc,0.186261,0.167167,0.168772,0.236579,0.227048,0.258980,2,0.161924,91e2f88dd6e3,...,0.43177,0.091813,0.097055,0.090208,0.022401,-0.074655,0.258980,0.161924,0.200962,0.036319
3,02996f1dd2ea,0.241699,0.260876,0.279610,0.383699,0.231447,0.284130,3,0.278804,bf5cccea572d,...,0.80323,0.023255,0.005326,0.004520,-0.099569,-0.104895,0.383699,0.231447,0.280038,0.046233
4,7c90be56d2ab,0.407423,0.431520,0.410924,0.420657,0.412563,0.430581,4,0.350284,ef50846afc0b,...,0.20797,-0.000938,0.080297,0.019657,0.009924,-0.070373,0.431520,0.350284,0.409136,0.025565


### Train scores

In [8]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if '_preds' in c]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE rnn01_preds:  0.2181183231819507
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE rnn27_preds:  0.21698040871120608


In [9]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [10]:
scores = []
for col in [c for c in preds_df.columns if '_preds' in c]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,rnn01_preds,0.219214,0.219456,0.216135,0.219705,0.215343
3,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
4,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
5,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
6,rnn27_preds,0.218097,0.218115,0.214982,0.218651,0.214379


### Correlations in test and train

In [11]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,rnn01_preds,mlp_preds,lgb31_preds,lgb02_preds,rnn27_preds
lgb27_preds,1.000000,0.915952,0.917540,0.919171,0.990750,0.988682,0.916804
rnn_preds,0.915952,1.000000,0.991623,0.929710,0.910602,0.913307,0.985360
rnn01_preds,0.917540,0.991623,1.000000,0.932528,0.912191,0.914714,0.985583
mlp_preds,0.919171,0.929710,0.932528,1.000000,0.914307,0.916343,0.925010
lgb31_preds,0.990750,0.910602,0.912191,0.914307,1.000000,0.991984,0.914204
lgb02_preds,0.988682,0.913307,0.914714,0.916343,0.991984,1.000000,0.916726
rnn27_preds,0.916804,0.985360,0.985583,0.925010,0.914204,0.916726,1.000000


In [12]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if '_preds' in c]].corr()

,lgb27_preds,rnn_preds,rnn01_preds,mlp_preds,lgb31_preds,lgb02_preds,rnn27_preds
lgb27_preds,1.000000,0.930288,0.930768,0.924283,0.991439,0.989502,0.930692
rnn_preds,0.930288,1.000000,0.993167,0.939951,0.927835,0.927867,0.988198
rnn01_preds,0.930768,0.993167,1.000000,0.942921,0.928527,0.928250,0.987039
mlp_preds,0.924283,0.939951,0.942921,1.000000,0.922413,0.921927,0.935069
lgb31_preds,0.991439,0.927835,0.928527,0.922413,1.000000,0.993168,0.931468
lgb02_preds,0.989502,0.927867,0.928250,0.921927,0.993168,1.000000,0.931447
rnn27_preds,0.930692,0.988198,0.987039,0.935069,0.931468,0.931447,1.000000


In [13]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [14]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [15]:
len(train_X)

1202739

In [27]:
import gc
gc.collect()

0

In [24]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    #max_bin=128,
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.220888	valid_1's rmse: 0.221204
[200]	training's rmse: 0.214674	valid_1's rmse: 0.215206
[300]	training's rmse: 0.213563	valid_1's rmse: 0.214281
[400]	training's rmse: 0.213226	valid_1's rmse: 0.214107
[500]	training's rmse: 0.213028	valid_1's rmse: 0.214046
[600]	training's rmse: 0.212866	valid_1's rmse: 0.214011
[700]	training's rmse: 0.212726	valid_1's rmse: 0.213986
[800]	training's rmse: 0.212593	valid_1's rmse: 0.213966
[900]	training's rmse: 0.212464	valid_1's rmse: 0.213949
[1000]	training's rmse: 0.212349	valid_1's rmse: 0.21394


KeyboardInterrupt: 

In [ ]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

In [ ]:
n_estimators = 1826
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

In [ ]:
%%time
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')

clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)

In [ ]:
test_df['deal_probability'] = clf.predict(test_df[cols])
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_0306AL2.csv.gz', compression='gzip', index=False, header=True)

In [ ]:
test_df[['item_id', 'deal_probability']].head()